In [34]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
import anndata as ad
import hdf5plugin
from sklearn.decomposition import TruncatedSVD
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os 
#external modules
#pretty plotting
import seaborn as sb
import sys 
import warnings
warnings.filterwarnings("ignore")
sb.set_context(context='poster')

import copy

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80)

scanpy==1.9.3 anndata==0.9.2 umap==0.5.3 numpy==1.24.4 scipy==1.10.1 pandas==2.0.3 scikit-learn==1.3.0 statsmodels==0.14.0 python-igraph==0.10.6 louvain==0.8.1 pynndescent==0.5.10


## Data loading

In [108]:
figure_path = r'C:\Users\ilosz01\OneDrive - Linköpings universitet\MarcinLab\SingleCellSequencing\scs_analysis\figuresR'
input_path = r'C:\Users\ilosz01\OneDrive - Linköpings universitet\MarcinLab\SingleCellSequencing\scs_analysis\out'


adata_umap_path = os.path.join(input_path,'control_scs_umap_from_R.h5ad')
adata_path = os.path.join(input_path,'campari_scs_from_R.h5ad')


NO_PCA = 20

In [109]:
adata_umap = ad.read_h5ad(adata_umap_path)

## EXPLORE

In [110]:
### Define list of relevant genes 

marker_genes = ['Trpm3','Trpm2','Smr2','Sstr2','Bmpr1b','Trpm8','Trpv1','Piezo2','Piezo1','Nppb',
                'Sst','Pvalb','Prokr2','Mrgprd','Mrgpra3','Cd34',
                'Th','Trpa1','Ntrk3','Ntrk2','Ntrk1','Ret','Tac1','Calca','Calcb','Nefh',
                'S100b','Scn10a','Slc17a8','Atf3','Pou4f3','Calb1','Calb2','Avil','Asic3',
                'Asic2','Asic1','Pou6f2','Avpr1a','Pou4f2','Sox10','Casq2','Chrna7','Chrna3',
                'P2rx3','Gfra2','Ldhb','Necab2','Spp1','Adm','Hpse','Adra2a']

In [134]:
# made some names shorter to display
marker_genes_dict = {
                    'RA-LTMR': ['Ntrk2'],
                    'C-LTMR': ['Cd34', 'Th'],
                    'Itch': ['Nppb','Mrgpra3'],
                    'Nonpept-Nocicept': ['Mrgprd'],
                    'Pept-Nocicept': ['Sstr2'],
                    'Interoceptors': ['Adra2a','Tac1'],
                    'A-HTMR': ['Smr2','Bmpr1b'],
                    'Propioceptors': ['Pvalb']
}
# marker_genes_dict = {
#                     'RA-LTMR': ['Ntrk2'],
#                     'C-LTMR': ['Cd34', 'Th'],
#                     'Itch': ['Nppb','Mrgpra3'],
#                     'Nonpeptidergic-Nociceptors': ['Mrgprd'],
#                     'Peptidergic-Nociceptors': ['Sstr2'],
#                     'Interoceptors': ['Adra2a','Tac1'],
#                     'A-HTMR': ['Smr2','Bmpr1b'],
#                     'Propioceptores': ['Pvalb']
# }

In [112]:
labels=['Control','pinch','new_stroke','ballon','AG','AD','heating','mock','old_stroke','anal_pinch','mock_w_poop','vaginal_distension','tomatoe','poop', 'bladder','CRD','AD+CFA','AGB+CFA','heating+CFA','tail brush','TRPM8 td tomatoe']
stim_str = ['0', '1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']

In [113]:
adata_umap

AnnData object with n_obs × n_vars = 1706 × 16000
    obs: 'barcode', 'stimulus', 'red', 'green', 'well_id', 'plate_number', 'batch', 'n_counts', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'louvain_r0.5', 'louvain_r1', 'louvain_r1.5', 'louvain_r2', 'louvain_r2.5'
    var: 'gene_id', 'gene_name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'log1p', 'louvain', 'louvain_r0.5_colors', 'louvain_r1.5_colors', 'louvain_r1_colors', 'louvain_r2.5_colors', 'louvain_r2_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [114]:
# remove some cells from data to be ranked (mock_w_poop,poop,CRD?)
stim2exclude = [10,13,14,15]
# Remove items with the value x from 'obs_variable'
for el in stim2exclude:
    adata_umap = adata_umap[adata_umap.obs['stimulus'] != el]

In [115]:
# # there is just one poop cell and it ruined gene ranking, so: replace 13=poop with 5=AD
# stims = list(adata_umap.obs['stimulus'].values)
# idx=stims.index(13)
# # replace 13=poop with 5=AD
# stims[idx] = 5
# adata_umap.obs['stimulus'] = stims

In [116]:
# Correct for ranking baloon=3 should be AD=5
stims = adata_umap.obs['stimulus'].values
stims[stims == 3] = 5
adata_umap.obs['stimulus'] = stims

In [117]:
# create category type obs variable that will be used for ranking
adata_umap.obs['stimulus_'] = adata_umap.obs['stimulus'].astype('category') # copy stimulus data to new obs for ranking 
new_values = [str(int(el)) for el in adata_umap.obs['stimulus_'].values]    # convert stimulus to string categories
adata_umap.obs['stimulus_'] = new_values                                    # replace float values with new string values (changed type to object)
adata_umap.obs['stimulus_'].astype('category')                              # make it category type again

0-sample1-sample12-sample123-sample1234     2
1-sample1-sample12-sample123-sample1234     2
2-sample1-sample12-sample123-sample1234     2
4-sample1-sample12-sample123-sample1234     1
5-sample1-sample12-sample123-sample1234     1
                                           ..
379-sample5                                18
380-sample5                                18
381-sample5                                18
382-sample5                                18
383-sample5                                18
Name: stimulus_, Length: 1689, dtype: category
Categories (16, object): ['0', '1', '11', '12', ..., '6', '7', '8', '9']

In [118]:
# create ordered list of stim names
stim_labels = []
for stim in adata_umap.obs['stimulus_'].values:
    idx = stim_str.index(stim)
    new_label = labels[idx]
    stim_labels.append(new_label)

In [119]:
adata_umap.obs['stimulus_'] = stim_labels 
adata_umap.obs['stimulus_']

0-sample1-sample12-sample123-sample1234     new_stroke
1-sample1-sample12-sample123-sample1234     new_stroke
2-sample1-sample12-sample123-sample1234     new_stroke
4-sample1-sample12-sample123-sample1234          pinch
5-sample1-sample12-sample123-sample1234          pinch
                                              ...     
379-sample5                                heating+CFA
380-sample5                                heating+CFA
381-sample5                                heating+CFA
382-sample5                                heating+CFA
383-sample5                                heating+CFA
Name: stimulus_, Length: 1689, dtype: object

In [120]:
sc.pp.log1p(adata_umap) # it was already logaritmized but adata.uns['log1p'] has no 'base' key
adata_umap.uns['log1p']

{'base': None}

In [121]:
sc.tl.rank_genes_groups(adata_umap, 'stimulus_', method='wilcoxon')

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:04)


In [122]:
pd.DataFrame(adata_umap.uns['rank_genes_groups']['names']).head(10)

,AD,AD+CFA,AG,AGB+CFA,Control,TRPM8 td tomatoe,anal_pinch,heating,heating+CFA,mock,new_stroke,old_stroke,pinch,tail brush,tomatoe,vaginal_distension
0,Ccn1,Gm29216,Mcam,Pfkm,Tmsb10,AA465934,Eef1a2,Gpm6a,Rbm3,Ptn,Ptn,campari2,Hmgb1,Pla2g7,1500015A07Rik,mt-Nd6
1,Neat1,Dync1h1,Tpbgl,Gm7308,Cystm1,Rps3a3,Rpl10-ps6,Skp1a,Ctsl,campari2,Rfk,cre,Uqcc2,Sdcbp,Vcp-rs,mt-Rnr2
2,campari2,Gm13394,Ak5,Ehd3,Pcsk1n,A930005H10Rik,Rpl10-ps5,Fnbp1l,Skp1a,Selenop,Nptn,Gm6063,Rhou,Aopep,Rps3a3,Foxn3
3,Nr4a1,Cltc,Atp2b2,Atp9a,Calcb,Gap43,Gm7476,Cd55,Eef1a1,Sult1a1,Sparc,Gm24447,Mbnl1,Gpx2,Gm13453,Rab5a
4,Slc9a3r1,Lars2,Map7d1,Syp,Mrpl57,Ntrk1,Kctd16,Adk,Srsf5,Rgcc,Rgcc,Eif2s3y,Fkbp1b,Mtfr1l,Gm20305,Vapa
5,Acaa1a,Akap12,Myo18a,Iars,Fam189b,Rbfox3,Atg9a,Cd82,Fgf13,cre,Eid1,Tmem176b,Bcl7c,Cd59a,Gm4202,Ptp4a2
6,Csmd1,Kif5a,Pygb,Gm13341,Nenf,Celf4,Phc3,Smim5,Morf4l2,Mt2,Cacybp,Gm17494,Ddx24,Manf,Ftl1-ps1,Marcks
7,cre,Scd2,Stac2,Nsf,Mt3,Jpt1,Prepl,Snx10,Basp1,Mbp,Trim2,Rn7sk,Galt,Pja1,Ap3s1,Pacsin1
8,Zfp36,Fasn,Scn8a,Rab15,Atpaf2,Zdhhc22,Samd12,Ifit3b,Npm1,Spp1,Pdha1,Rn7s2,Id1,mt-Cytb,Ptges3,Ppp2r5c
9,Id3,Syn1,Myh14,Flii,Map1b,Kit,Tom1l2,Moxd1,Mettl7a3,Ubc,Vegfa,Rps27,Ypel3,Pdcd10,Rpl9-ps6,Atxn7l3b


In [123]:
# there are some gene name duplicates
def unique(list1):
 
    # initialize a null list
    unique_list = []
    unique_index = []
 
    # traverse for all elements
    count = 0
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
            unique_index.append(count)
        count+=1
    return unique_list,unique_index
        
gene_name = []
for i in range(len(adata_umap.var['gene_name'])):
    gene_name.append(adata_umap.var['gene_name'].iloc[i])

# remove duplicates
unique_genes , unique_genes_index = unique(gene_name)

adata_umap = adata_umap[:,np.array(unique_genes_index)]

In [136]:
# show plot as pop-up
%matplotlib tk 
sc.pl.heatmap(adata_umap, marker_genes_dict, groupby='stimulus_', cmap='rainbow', use_raw=False, figsize=(10,10), 
              show=True,show_gene_labels=True,var_group_rotation=45, dendrogram = True,vmin=-3,vmax=3)

categories: AD, AD+CFA, AG, etc.
var_group_labels: RA-LTMR, C-LTMR, Itch, etc.
